In [1]:
import os
import time
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import seaborn as sns
from sklearn.utils.class_weight import compute_class_weight
import torch.nn.functional as F

In [2]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("GPU Count:", torch.cuda.device_count())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Force NVIDIA GPU (GPU 0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Verify GPU selection
print(f"Using device: {device}")
print("GPU Name:", torch.cuda.get_device_name(device))

# Create a random tensor and move it to the selected GPU
x = torch.randn(1000, 1000).to(device)
print("✅ Tensor successfully moved to:", x.device)

CUDA Available: True
GPU Count: 1
GPU Name: NVIDIA GeForce RTX 4070 Laptop GPU
Using device: cuda
Using device: cuda:0
GPU Name: NVIDIA GeForce RTX 4070 Laptop GPU
✅ Tensor successfully moved to: cuda:0


In [3]:
# Directories
data_dir = "E:/Works/12. Plant Diseases Classification/Datasets/Work_Dataset"
train_dir = os.path.join(data_dir, "train")
valid_dir = os.path.join(data_dir, "valid")
test_dir = os.path.join(data_dir, "test")
external_test_dir = os.path.join(data_dir, "External_testing_dataset")

In [4]:
# Hyperparameters
batch_size = 16
num_epochs = 50
learning_rate = 0.001
img_size = 224
weight_decay = 1e-4
patience = 5

In [5]:
# Transformations
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(img_size, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

valid_test_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [6]:
# Load Data
train_dataset = ImageFolder(train_dir, transform=train_transforms)
valid_dataset = ImageFolder(valid_dir, transform=valid_test_transforms)
test_dataset = ImageFolder(test_dir, transform=valid_test_transforms)
external_test_dataset = ImageFolder(external_test_dir, transform=valid_test_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
external_test_loader = DataLoader(external_test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
# Class Names
class_names = train_dataset.classes
num_classes = len(class_names)
print(f"Classes: {class_names}")

Classes: ['A_blk_rot_f', 'A_blk_spot', 'A_e_canker', 'A_gl_lf_spot', 'A_healthy_f', 'A_healthy_l', 'A_m_virus', 'Av_alg_lf_spot', 'Av_br_canker', 'Av_healthy_l', 'G_blk_spot_c', 'G_healthy_c', 'Kf_bac_canker', 'Kf_healthy_l', 'P_canker', 'P_fr_blight', 'P_healthy_f', 'P_healthy_l', 'P_s_pit', 'P_scab']


In [8]:
# Compute Class Weights
labels = np.array(train_dataset.targets)
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

In [9]:
# Define Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha  # Class weights
        self.gamma = gamma  # Focusing parameter
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction="none", weight=self.alpha)
        pt = torch.exp(-ce_loss)  # Probabilities of correct class
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss  # Focal loss formula

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [10]:
# Temperature Scaling Function
class TemperatureScaling(nn.Module):
    def __init__(self, init_temp=1.5):
        super(TemperatureScaling, self).__init__()
        self.temperature = nn.Parameter(torch.tensor(init_temp))

    def forward(self, logits):
        return logits / self.temperature

In [11]:
# Model Selection
models = {
    "ConvNeXt-Tiny": torchvision.models.convnext_tiny(weights=torchvision.models.ConvNeXt_Tiny_Weights.IMAGENET1K_V1),
    "EfficientNetV2": torchvision.models.efficientnet_v2_s(weights="IMAGENET1K_V1"),
    "RegNetY-8GF": torchvision.models.regnet_y_8gf(weights="IMAGENET1K_V1"),
    "MaxViT": torchvision.models.maxvit_t(weights="IMAGENET1K_V1"),
}

E:\Works\12. Plant Diseases Classification\venv\lib\site-packages\torch\functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3596.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [12]:
# Modify models for classification
for name, model in models.items():
    if "convnext" in name.lower():
        model.classifier[2] = nn.Linear(model.classifier[2].in_features, num_classes)
    elif "efficientnet" in name.lower():
        in_features = model.classifier[-1].in_features  
        model.classifier[-1] = nn.Linear(in_features, num_classes)
    elif "regnet" in name.lower():
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif "maxvit" in name.lower():
        in_features = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(in_features, num_classes)

    # Add Temperature Scaling
    model.temperature_scaling = TemperatureScaling()
    
    model.to(device)

In [13]:
# Loss Function: Choose between Class Weighted Loss or Focal Loss
use_focal_loss = True  # Set to False to use standard class-weighted cross-entropy

if use_focal_loss:
    criterion = FocalLoss(alpha=class_weights)
else:
    criterion = nn.CrossEntropyLoss(weight=class_weights)

# Optimizers with Weight Decay
optimizers = {name: optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay) 
              for name, model in models.items()}

In [14]:
# Train Function
def train_model(model, optimizer, train_loader, valid_loader, num_epochs, device):
    history = {"train_loss": [], "train_acc": [], "valid_loss": [], "valid_acc": []}
    start_time = time.time()
    
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)
    best_val_loss = float("inf")
    epochs_without_improvement = 0
    criterion = nn.CrossEntropyLoss(weight=class_weights)  # Use computed class weights

    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            outputs = model.temperature_scaling(outputs)  # Apply temperature scaling
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / len(train_loader)
        train_acc = correct / total

        # Validation
        model.eval()
        val_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                outputs = model.temperature_scaling(outputs)  # Apply temperature scaling
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        val_loss /= len(valid_loader)
        val_acc = correct / total

        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["valid_loss"].append(val_loss)
        history["valid_acc"].append(val_acc)

        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print("Early stopping triggered.")
                break

    total_time = time.time() - start_time
    return history, total_time

In [15]:
# Store Results
results = {}
training_times = {}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for name, model in models.items():
    model.to(device)  # Move model to GPU if available
    print(f"\nTraining {name}...")
    history, total_time = train_model(model, optimizers[name], train_loader, valid_loader, num_epochs, device)
    results[name] = {"model": model, "history": history}
    
    # Store the training time for each model in the dictionary
    training_times[name] = total_time  # Using model name as the key and storing the training time

print("\nTraining Time per Model:")
for name, duration in training_times.items():
    print(f"{name}: {duration:.2f} seconds ({duration/60:.2f} minutes)")


Training ConvNeXt-Tiny...


E:\Works\12. Plant Diseases Classification\venv\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/50 - Train Loss: 2.9416, Train Acc: 0.0781, Val Loss: 2.6280, Val Acc: 0.1135
Epoch 2/50 - Train Loss: 2.4985, Train Acc: 0.1748, Val Loss: 2.0951, Val Acc: 0.3233
Epoch 3/50 - Train Loss: 2.1250, Train Acc: 0.2529, Val Loss: 1.9784, Val Acc: 0.3261
Epoch 4/50 - Train Loss: 1.8958, Train Acc: 0.3277, Val Loss: 1.8258, Val Acc: 0.3448
Epoch 5/50 - Train Loss: 1.8000, Train Acc: 0.3713, Val Loss: 1.7632, Val Acc: 0.3606
Epoch 6/50 - Train Loss: 1.5770, Train Acc: 0.4408, Val Loss: 1.5369, Val Acc: 0.4626
Epoch 7/50 - Train Loss: 1.5364, Train Acc: 0.4638, Val Loss: 1.5312, Val Acc: 0.4626
Epoch 8/50 - Train Loss: 1.4269, Train Acc: 0.4844, Val Loss: 1.3547, Val Acc: 0.5115
Epoch 9/50 - Train Loss: 1.4093, Train Acc: 0.4877, Val Loss: 1.3685, Val Acc: 0.5057
Epoch 10/50 - Train Loss: 1.3187, Train Acc: 0.5243, Val Loss: 1.4318, Val Acc: 0.5129
Epoch 11/50 - Train Loss: 1.2914, Train Acc: 0.5374, Val Loss: 1.4892, Val Acc: 0.4799
Epoch 12/50 - Train Loss: 1.1115, Train Acc: 0.5970,

In [16]:
# Create a folder to save the plots
save_dir = "E:/Works/12. Plant Diseases Classification/Results/10. Testing Time/4. Testing Time without Metahurestics/Accuracy_Loss_Curves"
os.makedirs(save_dir, exist_ok=True)

# Modify plot_metrics to save plots
def plot_metrics(history, model_name):
    epochs = range(1, len(history["train_acc"]) + 1)  # Fix: Use actual history length
    plt.figure(figsize=(12, 5))

    # Accuracy Plot
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history["train_acc"], label="Train Accuracy")
    plt.plot(epochs, history["valid_acc"], label="Validation Accuracy")
    plt.title(f"{model_name} Accuracy")
    plt.legend()

    # Loss Plot
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history["train_loss"], label="Train Loss")
    plt.plot(epochs, history["valid_loss"], label="Validation Loss")
    plt.title(f"{model_name} Loss")
    plt.legend()

    # Save the plot
    plt.savefig(os.path.join(save_dir, f"{model_name}_metrics.png"))
    plt.close()

# Plot for each model and save
for name, data in results.items():
    plot_metrics(data["history"], name)

In [17]:
# Function to evaluate model and save classification report
def evaluate_model(model, test_loader, model_name, dataset_name):
    model.eval()
    y_true, y_pred = [], []
    
    start_time = time.time()  # Start timing

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    end_time = time.time()  # End timing
    testing_time = round((end_time - start_time) * 1000, 2)  # Convert to milliseconds

    report = classification_report(y_true, y_pred, target_names=class_names, digits=4, output_dict=True)
    print(f"\nClassification Report for {model_name} on {dataset_name}:\n")
    print(classification_report(y_true, y_pred, target_names=class_names, digits=4))
    
    return y_true, y_pred, report, testing_time

    

# Evaluate each model on both test sets and store results
model_reports = {}
predictions = {}
testing_times = {}

for mh_name, mh_results in results.items():  # Iterate over models
    model = mh_results["model"]  # Directly retrieve the model from the 'model' key in mh_results

    # Evaluate on Original Test Set
    y_true, y_pred, report, test_time = evaluate_model(model, test_loader, f"{mh_name}", "Original Test Set")
    model_reports[f"{mh_name}_Test"] = report
    predictions[f"{mh_name}_Test"] = (y_true, y_pred)
    testing_times[f"{mh_name}_Test"] = test_time

    # Evaluate on External Test Set
    y_true, y_pred, report, test_time = evaluate_model(model, external_test_loader, f"{mh_name}", "External Test Set")
    model_reports[f"{mh_name}_External_Test"] = report
    predictions[f"{mh_name}_External_Test"] = (y_true, y_pred)
    testing_times[f"{mh_name}_External_Test"] = test_time


Classification Report for ConvNeXt-Tiny on Original Test Set:

                precision    recall  f1-score   support

   A_blk_rot_f     0.3636    0.6667    0.4706         6
    A_blk_spot     0.3571    0.5000    0.4167        10
    A_e_canker     0.6818    0.8824    0.7692        17
  A_gl_lf_spot     0.7692    0.6452    0.7018        31
   A_healthy_f     0.7778    0.8235    0.8000        17
   A_healthy_l     0.2105    0.2353    0.2222        17
     A_m_virus     0.6316    0.6000    0.6154        20
Av_alg_lf_spot     0.4706    1.0000    0.6400         8
  Av_br_canker     0.9048    0.7600    0.8261        25
  Av_healthy_l     0.9231    0.5217    0.6667        23
  G_blk_spot_c     0.5833    0.8750    0.7000         8
   G_healthy_c     1.0000    0.7647    0.8667        17
 Kf_bac_canker     0.9677    0.9375    0.9524        32
  Kf_healthy_l     0.9643    1.0000    0.9818        27
      P_canker     0.6667    0.8000    0.7273        10
   P_fr_blight     0.6667    0.3333    

E:\Works\12. Plant Diseases Classification\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
E:\Works\12. Plant Diseases Classification\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
E:\Works\12. Plant Diseases Classification\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap


Classification Report for ConvNeXt-Tiny on External Test Set:

                precision    recall  f1-score   support

   A_blk_rot_f     0.4306    0.4921    0.4593        63
    A_blk_spot     0.1154    0.4000    0.1791        15
    A_e_canker     0.1111    0.0833    0.0952        12
  A_gl_lf_spot     0.3077    0.2667    0.2857        15
   A_healthy_f     0.8125    0.2955    0.4333        44
   A_healthy_l     0.0000    0.0000    0.0000        15
     A_m_virus     0.5294    0.2432    0.3333        37
Av_alg_lf_spot     0.0385    0.0909    0.0541        11
  Av_br_canker     0.2308    0.1579    0.1875        19
  Av_healthy_l     0.4706    0.2667    0.3404        30
  G_blk_spot_c     0.1111    0.2000    0.1429         5
   G_healthy_c     0.9615    0.5882    0.7299        85
 Kf_bac_canker     0.0000    0.0000    0.0000        14
  Kf_healthy_l     0.6667    0.0571    0.1053        35
      P_canker     0.2188    0.6364    0.3256        11
   P_fr_blight     0.2000    0.5000    

In [18]:
# Create a folder to save the confusion matrices
cm_save_dir = "E:/Works/12. Plant Diseases Classification/Results/10. Testing Time/4. Testing Time without Metahurestics/Confusion Matrix"
os.makedirs(cm_save_dir, exist_ok=True)

# Modify plot_confusion_matrix to save plots
def plot_confusion_matrix(y_true, y_pred, classes, model_name, dataset_name):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt="d", xticklabels=classes, yticklabels=classes, cmap="Blues")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title(f"Confusion Matrix - {model_name} ({dataset_name})")

    # Save the plot with dataset name included
    filename = f"{model_name}_{dataset_name.replace(' ', '_')}_confusion_matrix.png"
    plt.savefig(os.path.join(cm_save_dir, filename))
    plt.close()

# Generate and save confusion matrices for both test sets
for name, data in results.items():
    # Confusion Matrix for Original Test Set
    y_true, y_pred = predictions[f"{name}_Test"]
    plot_confusion_matrix(y_true, y_pred, class_names, name, "Original_Test_Set")

    # Confusion Matrix for External Test Set
    y_true, y_pred = predictions[f"{name}_External_Test"]
    plot_confusion_matrix(y_true, y_pred, class_names, name, "External_Test_Set")

In [19]:
# Create a folder to save the ROC curves
roc_save_dir = "E:/Works/12. Plant Diseases Classification/Results/10. Testing Time/4. Testing Time without Metahurestics/ROC Curves"
os.makedirs(roc_save_dir, exist_ok=True)

# Modify compute_auc to save plots
def compute_auc(model, test_loader, model_name, dataset_name):
    model.eval()
    y_true, y_scores = [], []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)

            y_true.extend(labels.cpu().numpy())
            y_scores.extend(probabilities.cpu().numpy())

    y_true = np.array(y_true)
    y_scores = np.array(y_scores)

    auc_scores = {}
    plt.figure(figsize=(10, 7))

    for i in range(num_classes):
        fpr, tpr, _ = roc_curve(y_true == i, y_scores[:, i])
        roc_auc = auc(fpr, tpr)
        auc_scores[class_names[i]] = round(roc_auc, 4)  # Round AUC to 4 decimal points
        plt.plot(fpr, tpr, label=f"{model_name} (Class {class_names[i]}) AUC = {roc_auc:.4f}")

    plt.plot([0, 1], [0, 1], "k--")  # Diagonal line
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"ROC Curve - {model_name} ({dataset_name})")
    plt.legend(loc="lower right")

    # Save the plot with dataset name included
    filename = f"{model_name}_{dataset_name.replace(' ', '_')}_roc_curve.png"
    plt.savefig(os.path.join(roc_save_dir, filename))
    plt.close()

    return auc_scores

# Compute and store AUC scores for each model and both test sets
auc_results = {}

for name, data in results.items():
    # AUC for Original Test Set
    auc_scores_test = compute_auc(data["model"], test_loader, name, "Original_Test_Set")
    auc_results[f"{name}_Test"] = auc_scores_test

    # AUC for External Test Set
    auc_scores_external_test = compute_auc(data["model"], external_test_loader, name, "External_Test_Set")
    auc_results[f"{name}_External_Test"] = auc_scores_external_test

In [20]:
# Extract relevant metrics separately for both test sets
model_names = []
dataset_types = []
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
auc_mean_scores = []
training_times_list = []  # Store training times in this list
testing_time_values = []

for name, report in model_reports.items():
    dataset_type = "External Test Set" if "External_Test" in name else "Original Test Set"
    base_name = name.replace("_Test", "").replace("_External_Test", "")

    model_names.append(base_name)
    dataset_types.append(dataset_type)
    
    accuracy_scores.append(round(report["accuracy"] * 100, 2))  # Convert to percentage
    precision_scores.append(round(report["macro avg"]["precision"] * 100, 2))
    recall_scores.append(round(report["macro avg"]["recall"] * 100, 2))
    f1_scores.append(round(report["macro avg"]["f1-score"] * 100, 2))
    
    # Compute mean AUC across all classes (ensure auc_results is correctly defined and used)
    mean_auc = np.mean(list(auc_results.get(name, {}).values()))  # Use .get() to avoid errors
    auc_mean_scores.append(round(mean_auc * 100, 2))

    # Correct way to extract training time from training_times dictionary
    training_time = training_times.get(base_name, 0)  # .get() works now because training_times is a dictionary
    training_times_list.append(round(training_time / 60, 2))  # Store in list, convert to minutes

    # Extract testing time from testing_times dictionary
    testing_time = testing_times.get(name, 0)  # Use .get() to safely retrieve testing time
    testing_time_values.append(testing_time)  # Store in milliseconds

# Create DataFrame with the extracted metrics
df_metrics = pd.DataFrame({
    "Model": model_names,
    "Dataset": dataset_types,
    "Accuracy (%)": accuracy_scores,
    "Precision (%)": precision_scores,
    "Recall (%)": recall_scores,
    "F1-Score (%)": f1_scores,
    "Mean AUC (%)": auc_mean_scores,
    "Training Time (mins)": training_times_list,
    "Testing Time (ms)": testing_time_values
})

# Display the table
print("\nModel Performance Comparison for Both Test Sets:\n")
print(df_metrics)

# Save DataFrame to CSV
csv_path = "E:/Works/12. Plant Diseases Classification/Results/10. Testing Time/4. Testing Time without Metahurestics/model_performance_metrics_without_metahurestics.csv"
df_metrics.to_csv(csv_path, index=False)


Model Performance Comparison for Both Test Sets:

                     Model            Dataset  Accuracy (%)  Precision (%)  \
0            ConvNeXt-Tiny  Original Test Set         71.84          70.86   
1   ConvNeXt-Tiny_External  External Test Set         33.46          30.83   
2           EfficientNetV2  Original Test Set         90.80          89.75   
3  EfficientNetV2_External  External Test Set         67.31          65.85   
4              RegNetY-8GF  Original Test Set         92.53          91.91   
5     RegNetY-8GF_External  External Test Set         54.42          57.32   
6                   MaxViT  Original Test Set         89.66          89.69   
7          MaxViT_External  External Test Set         67.50          71.24   

   Recall (%)  F1-Score (%)  Mean AUC (%)  Training Time (mins)  \
0       69.50         67.43         98.05                 24.22   
1       26.26         23.96         84.82                  0.00   
2       89.53         88.85         99.76    